# LAB 5: ONNX
### AI and Machine Learning // Suchkova Natalia М8О - 114М - 22
16.06.23 @ MAI IT-Center

### The Task
1. Обучить нейросеть 
2. Pruning 
3. Конвертировать в ONNX
4. Запустить в Unity

<a id='g0'></a>
## Table of Contents


1. [**Что такое гомография**](#g1)


2. [**Делаем панораму**](#g2)


3. [**Что такое триангуляция**](#g3)


4. [**Используем триангуляцию**](#g4)
    - [**RANSAC**](#g41)
    - [**WARP**](#g42)
    - [**Смотрим исходные фото**](#g43)
    - [**применяем и смотрим результаты**](#g44)
    

ONNX (Open Neural Network Exchange) — открытая библиотека программного обеспечения для построения нейронных сетей глубокого обучения. С помощью ONNX ИИ-разработчики могут обмениваться моделями между различными инструментами и выбирать наилучшую комбинацию этих инструментов. ONNX разрабатывается и поддерживается совместно компаниями Microsoft, Facebook, Amazon и другими партнерами как проект с открытым исходным кодом